In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import gensim.downloader as api

In [2]:
class Net(nn.Module):
  def __init__(self, num_words, emb_dim, num_y, embeds=None):
    super().__init__()
    print(num_words)
    print(emb_dim)
    print(num_y)
    self.emb = nn.Embedding(num_words, emb_dim)
    if embeds is not None:
      self.emb.weight = nn.Parameter(torch.Tensor(embeds))
    self.linear = nn.Linear(emb_dim, num_y)
    self.sigmoid = nn.Sigmoid()

  def forward(self, text):
    embeds = torch.mean(self.emb(text), dim=0)
    print(embeds)
    return self.sigmoid(self.linear(embeds))

In [3]:
def load_vocab(text):
  word_to_ix = {}
  for sent, label in text:
    for word in sent.split():
      word_to_ix.setdefault(word, len(word_to_ix))
  # print(word_to_ix)
  return word_to_ix


In [5]:
train_data = [("I am doing good", 1), ("this is not right", 0)]
tok_to_ix = load_vocab(train_data)
print(tok_to_ix)

{'I': 0, 'am': 1, 'doing': 2, 'good': 3, 'this': 4, 'is': 5, 'not': 6, 'right': 7}


In [6]:
num_classes = 1
learning_rate = 0.0001
emb_dim = 50
embeds = None

# embeds = api.load('glove-twitter-25').vectors # Change to 'word2vec-google-news-300' for word2vec or 'glove-wiki-gigaword-100' for Wikipedia-trained Glove
# emb_dim = embeds.shape[1]

model = Net(len(tok_to_ix), emb_dim, num_classes, embeds)
print(model)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()

8
50
1
Net(
  (emb): Embedding(8, 50)
  (linear): Linear(in_features=50, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [7]:
n_epochs = 10
for epoch in range(n_epochs):
  model.train()
  for text, label in train_data:
    print(text)
    x = [tok_to_ix[tok] for tok in text.split()]
    x_train_tensor = torch.LongTensor(x)
    y_train_tensor = torch.Tensor([label])
    pred_y = model(x_train_tensor)
    loss = loss_fn(pred_y, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print("\nEpoch:", epoch)
  print("Training loss:", loss.item())

I am doing good
tensor([-0.1267, -0.3910, -0.0229,  0.4649, -0.1008,  0.2766, -0.2146, -0.5575,
         0.6730,  0.4272, -0.0530,  0.0225,  0.5759, -0.5417, -0.0534, -0.3583,
        -0.4070, -0.3552,  0.4619, -0.2058, -0.8188,  0.9629, -0.3005, -0.9509,
         0.1333,  0.1298,  0.2234, -0.1022,  0.9906, -1.0238,  0.2084,  0.2539,
         0.0057,  0.9154,  0.8892, -0.2244,  0.3936,  0.5331,  0.1475,  0.2159,
         0.1005, -0.4759,  0.0810,  0.8932, -0.0604, -0.4768, -0.5771, -0.6262,
        -0.0260, -0.3744], grad_fn=<MeanBackward1>)
this is not right
tensor([-0.5630, -1.1594,  0.2562,  0.4998,  0.4587, -0.4231, -0.3306,  0.1650,
        -0.7200, -0.3271, -0.8323, -0.5915, -0.1571,  0.4647, -0.5104,  0.3359,
         0.3024, -0.1218,  0.2846,  0.0443, -0.4409, -0.2516, -0.2544,  0.2513,
        -0.1352,  0.2772,  0.4354,  0.5196, -0.3432,  0.2859,  0.4212, -0.2957,
         0.1819, -0.0728,  0.1215, -0.1898,  0.4586, -0.1297, -0.6502,  0.1970,
        -0.4743,  0.4746, -0.3258,